In [5]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install findspark
import findspark
findspark.init()
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import Row
from pyspark.sql.functions import col

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
spark = SparkSession.builder.appName('Dharaneesh').getOrCreate()

In [7]:
raw_data= spark.read.option("header", "true").option("mode", "DROPMALFORMED").option("inferSchema", 'True').csv('/content/AppleStore.csv')

In [8]:
raw_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- size_bytes: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- price: double (nullable = true)
 |-- rating_count_tot: integer (nullable = true)
 |-- rating_count_ver: integer (nullable = true)
 |-- user_rating: double (nullable = true)
 |-- user_rating_ver: double (nullable = true)
 |-- ver: string (nullable = true)
 |-- cont_rating: string (nullable = true)
 |-- prime_genre: string (nullable = true)
 |-- sup_devices.num: integer (nullable = true)
 |-- ipadSc_urls.num: integer (nullable = true)
 |-- lang.num: integer (nullable = true)
 |-- vpp_lic: integer (nullable = true)



Q1. The app size is provided to us in bytes. Create a new column 'AppSizeMbs'Convert the bytes to MB's

In [10]:
data= raw_data.withColumn("AppSizeMbs",raw_data.size_bytes/1000000)
data.show(5)

+---------+--------------------+----------+--------+-----+----------------+----------------+-----------+---------------+------+-----------+------------+---------------+---------------+--------+-------+----------+
|       id|            app_name|size_bytes|currency|price|rating_count_tot|rating_count_ver|user_rating|user_rating_ver|   ver|cont_rating| prime_genre|sup_devices.num|ipadSc_urls.num|lang.num|vpp_lic|AppSizeMbs|
+---------+--------------------+----------+--------+-----+----------------+----------------+-----------+---------------+------+-----------+------------+---------------+---------------+--------+-------+----------+
|281656475|     PAC-MAN Premium| 100788224|     USD| 3.99|           21292|              26|        4.0|            4.5| 6.3.5|         4+|       Games|             38|              5|      10|      1|100.788224|
|281796108|Evernote - stay o...| 158578688|     USD|  0.0|          161065|              26|        4.0|            3.5| 8.2.2|         4+|Productiv

In [11]:
data.toPandas().head(5)

,id,app_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic,AppSizeMbs
0,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1,100.788224
1,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1,158.578688
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1,100.524032
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1,128.512000
4,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1,92.774400


Q.2 List the top 20 most rated apps. Make sure to show full app name.

In [12]:
data.select('app_name', 'rating_count_tot').sort(data.rating_count_tot.desc()).show(20,False)

+-----------------------------------+----------------+
|app_name                           |rating_count_tot|
+-----------------------------------+----------------+
|Facebook                           |2974676         |
|Instagram                          |2161558         |
|Clash of Clans                     |2130805         |
|Temple Run                         |1724546         |
|Pandora - Music & Radio            |1126879         |
|Pinterest                          |1061624         |
|Bible                              |985920          |
|Candy Crush Saga                   |961794          |
|Spotify Music                      |878563          |
|Angry Birds                        |824451          |
|Subway Surfers                     |706110          |
|Fruit Ninja Classic                |698516          |
|Solitaire                          |679055          |
|CSR Racing                         |677247          |
|Crossy Road - Endless Arcade Hopper|669079          |
|Injustice

Q.3 What % of the apps has multiple language support.

In [13]:
data = data.toDF(*(c.replace('.', '_') for c in data.columns))
data.filter(data.lang_num > 1).count() * 100 / data.count()

47.08906488814784

Datasets for Q.4

In [14]:
aisles = spark.read.option("header", "true").option("inferSchema", 'True').csv("/content/aisles.csv")

In [15]:
departments = spark.read.option("header", "true").option("inferSchema", 'true').csv('/content/departments.csv')

In [16]:
products = spark.read.option("header", "true").option("inferSchema", 'True').csv("/content/products.csv")

In [17]:
orders = spark.read.option("header", "true").option("inferSchema", 'True').csv("/content/orders.csv")

In [21]:
orders_train = spark.read.option("header","true").option('inferSchema','true').csv('/content/order_products__train.csv')

Q4. Create a single table joining all the dataframes using common keys and print the number of observations.

In [22]:
data= products.join(aisles, aisles.aisle_id == products.aisle_id,"left").drop(aisles.aisle_id).join(departments, departments.department_id == products.department_id,"left").drop(departments.department_id).join(orders_train, orders_train.product_id == products.product_id,"right").drop(orders_train.product_id).join(orders, orders.order_id == orders_train.order_id, "left").drop(orders.order_id)
data.count()

107205

Q5. List the top 20 ordered products.

In [23]:
data.groupBy("product_name").count().sort(col("count").desc()).show(20,False)

+----------------------+-----+
|product_name          |count|
+----------------------+-----+
|Banana                |1423 |
|Bag of Organic Bananas|1220 |
|Organic Strawberries  |833  |
|Organic Baby Spinach  |752  |
|Large Lemon           |634  |
|Organic Avocado       |593  |
|Organic Hass Avocado  |541  |
|Strawberries          |511  |
|Limes                 |444  |
|Organic Raspberries   |411  |
|Organic Whole Milk    |394  |
|Organic Cucumber      |372  |
|Organic Blueberries   |364  |
|Organic Zucchini      |340  |
|Seedless Red Grapes   |321  |
|Organic Grape Tomatoes|319  |
|Organic Garlic        |319  |
|Organic Yellow Onion  |318  |
|Organic Red Onion     |306  |
|Organic Baby Carrots  |296  |
+----------------------+-----+
only showing top 20 rows



Q6. List top 20 products ordered between 8 am to 11 am.

In [24]:
data.filter("order_hour_of_day >= 8 and order_hour_of_day<=11").groupBy("product_name").count().sort(col("count").desc()).show(20,False)

+---------------------------------+-----+
|product_name                     |count|
+---------------------------------+-----+
|Banana                           |24   |
|Organic Strawberries             |17   |
|Bag of Organic Bananas           |12   |
|Large Lemon                      |12   |
|Organic Baby Spinach             |9    |
|Broccoli Crown                   |8    |
|Yellow Onions                    |8    |
|Organic Gala Apples              |7    |
|Organic Avocado                  |6    |
|Organic Yellow Onion             |6    |
|Organic Grape Tomatoes           |6    |
|Organic Cucumber                 |6    |
|Seedless Red Grapes              |6    |
|Organic Unsweetened Almond Milk  |5    |
|Limes                            |5    |
|Organic Baby Arugula             |5    |
|Organic Peeled Whole Baby Carrots|5    |
|Organic Raspberries              |5    |
|Organic Blueberries              |5    |
|Organic Romaine Lettuce          |5    |
+---------------------------------